In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, ArrayType, LongType, DoubleType, FloatType, DataType, DateType, TimestampType
from pyspark.sql.functions import regexp_replace,row_number
from pyspark.sql.window import Window

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed')).History will not be written to the database.


In [2]:
spark = (SparkSession
 .builder
 .appName("City_State")
 .getOrCreate())

22/08/27 05:51:58 WARN Utils: Your hostname, ubuntu resolves to a loopback address: 127.0.1.1; using 192.168.197.132 instead (on interface ens33)
22/08/27 05:51:58 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/08/27 05:52:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/08/27 05:52:08 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/08/27 05:52:08 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [23]:
schema = StructType([
             StructField('City', StringType(), True),
             StructField('State', StringType(), True)
         ])

In [24]:
df =  spark.read.option('header','True').schema(schema).option('sep',',').csv('German_Cities_And_State.csv')

In [25]:
df.printSchema()

root
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)



In [32]:
df.show()

+--------------------+--------------------+
|                City|               State|
+--------------------+--------------------+
|              Berlin|              Berlin|
|              Berlin|              Berlin|
|             Hamburg|             Hamburg|
|    Munich (München)|             Bavaria|
|      Cologne (Köln)|North Rhine-Westp...|
|   Frankfurt am Main|               Hesse|
|           Stuttgart|   Baden-Württemberg|
|          Düsseldorf|North Rhine-Westp...|
|            Dortmund|North Rhine-Westp...|
|               Essen|North Rhine-Westp...|
|             Leipzig|              Saxony|
|              Bremen|              Bremen|
|             Dresden|              Saxony|
|  Hanover (Hannover)|        Lower Saxony|
|Nuremberg (Nürnberg)|             Bavaria|
|            Duisburg|North Rhine-Westp...|
|              Bochum|North Rhine-Westp...|
|           Wuppertal|North Rhine-Westp...|
|           Bielefeld|North Rhine-Westp...|
|                Bonn|North Rhin

In [33]:
df = df.dropDuplicates()

In [34]:
df.show()

+-----------------+--------------------+
|             City|               State|
+-----------------+--------------------+
|        Darmstadt|               Hesse|
|       Reutlingen|   Baden-Württemberg|
|       Hildesheim|        Lower Saxony|
|          Krefeld|North Rhine-Westp...|
|           Kassel|               Hesse|
|           Siegen|North Rhine-Westp...|
|        Magdeburg|       Saxony-Anhalt|
|          Leipzig|              Saxony|
|    Halle (Saale)|       Saxony-Anhalt|
|          Potsdam|         Brandenburg|
|              Ulm|   Baden-Württemberg|
|       Regensburg|             Bavaria|
|Offenbach am Main|               Hesse|
|        Pforzheim|   Baden-Württemberg|
|       Düsseldorf|North Rhine-Westp...|
| Munich (München)|             Bavaria|
|        Oldenburg|        Lower Saxony|
|         Chemnitz|              Saxony|
|       Leverkusen|North Rhine-Westp...|
|             Hamm|North Rhine-Westp...|
+-----------------+--------------------+
only showing top

In [35]:
window = Window.orderBy('State')
df = df.withColumn('Id',row_number().over(window))

df.show(truncate=False)

22/08/27 06:35:46 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/27 06:35:46 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/27 06:35:47 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/27 06:35:47 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+--------------------+-----------------+---+
|City                |State            |Id |
+--------------------+-----------------+---+
|Reutlingen          |Baden-Württemberg|1  |
|Ulm                 |Baden-Württemberg|2  |
|Pforzheim           |Baden-Württemberg|3  |
|Stuttgart           |Baden-Württemberg|4  |
|Karlsruhe           |Baden-Württemberg|5  |
|Freiburg im Breisgau|Baden-Württemberg|6  |
|Heilbronn           |Baden-Württemberg|7  |
|Heidelberg          |Baden-Württemberg|8  |
|Mannheim            |Baden-Württemberg|9  |
|Regensburg          |Bavaria          |10 |
|Munich (München)    |Bavaria          |11 |
|Nuremberg (Nürnberg)|Bavaria          |12 |
|Würzburg            |Bavaria          |13 |
|Ingolstadt          |Bavaria          |14 |
|Fürth               |Bavaria          |15 |
|Augsburg            |Bavaria          |16 |
|Erlangen            |Bavaria          |17 |
|Berlin              |Berlin           |18 |
|Potsdam             |Brandenburg      |19 |
|Bremen   

In [36]:
df.write.format('csv').option('header','true').mode('overwrite').save('State')

22/08/27 06:36:12 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/27 06:36:12 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/27 06:36:13 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/27 06:36:13 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
